# **Extracting Information from Legal Documents Using RAG**

## **Objective**

The main objective of this assignment is to process and analyse a collection text files containing legal agreements (e.g., NDAs) to prepare them for implementing a **Retrieval-Augmented Generation (RAG)** system. This involves:

* Understand the Cleaned Data : Gain a comprehensive understanding of the structure, content, and context of the cleaned dataset.
* Perform Exploratory Analysis : Conduct bivariate and multivariate analyses to uncover relationships and trends within the cleaned data.
* Create Visualisations : Develop meaningful visualisations to support the analysis and make findings interpretable.
* Derive Insights and Conclusions : Extract valuable insights from the cleaned data and provide clear, actionable conclusions.
* Document the Process : Provide a detailed description of the data, its attributes, and the steps taken during the analysis for reproducibility and clarity.

The ultimate goal is to transform the raw text data into a clean, structured, and analysable format that can be effectively used to build and train a RAG system for tasks like information retrieval, question-answering, and knowledge extraction related to legal agreements.

### **Business Value**  


The project aims to leverage RAG to enhance legal document processing for businesses, law firms, and regulatory bodies. The key business objectives include:

* Faster Legal Research: <br> Reduce the time lawyers and compliance officers spend searching for relevant case laws, precedents, statutes, or contract clauses.
* Improved Contract Analysis: <br> Automatically extract key terms, obligations, and risks from lengthy contracts.
* Regulatory Compliance Monitoring: <br> Help businesses stay updated with legal and regulatory changes by retrieving relevant legal updates.
* Enhanced Decision-Making: <br> Provide accurate and context-aware legal insights to assist in risk assessment and legal strategy.


**Use Cases**
* Legal Chatbots
* Contract Review Automation
* Tracking Regulatory Changes and Compliance Monitoring
* Case Law Analysis of past judgments
* Due Diligence & Risk Assessment

## **1. Data Loading, Preparation and Analysis** <font color=red> [20 marks] </font><br>

### **1.1 Data Understanding**

The dataset contains legal documents and contracts collected from various sources. The documents are present as text files (`.txt`) in the *corpus* folder.

There are four types of documents in the *courpus* folder, divided into four subfolders.
- `contractnli`: contains various non-disclosure and confidentiality agreements
- `cuad`: contains contracts with annotated legal clauses
- `maud`: contains various merger/acquisition contracts and agreements
- `privacy_qa`: a question-answering dataset containing privacy policies

The dataset also contains evaluation files in JSON format in the *benchmark* folder. The files contain the questions and their answers, along with sources. For each of the above four folders, there is a `json` file: `contractnli.json`, `cuad.json`, `maud.json` `privacy_qa.json`. The file structure is as follows:

```
{
    "tests": [
        {
            "query": <question1>,
            "snippets": [{
                    "file_path": <source_file1>,
                    "span": [ begin_position, end_position ],
                    "answer": <relevant answer to the question 1>
                },
                {
                    "file_path": <source_file2>,
                    "span": [ begin_position, end_position ],
                    "answer": <relevant answer to the question 2>
                }, ....
            ]
        },
        {
            "query": <question2>,
            "snippets": [{<answer context for que 2>}]
        },
        ... <more queries>
    ]
}
```

# RAG Legal Docs Assignment
#####**Name:** Rahul Kumar, Prabhjyot Singh, and Mohan Nishantam
**Assignment Title:** Retrieval-Augmented Generation for Legal Documents  

### **1.2 Load and Preprocess the data** <font color=red> [5 marks] </font><br>

#### Loading libraries

In [ ]:
## The following libraries might be useful
# !pip install -q langchain-openai
# !pip install -U -q langchain-community
# !pip install -U -q langchain-chroma
# !pip install -U -q datasets
# !pip install -U -q ragas
# !pip install -U -q rouge_score

In [ ]:
# Import essential libraries
!pip install langchain llama-index openai rouge-score ragas evaluate nltk scikit-learn tiktoken


#### **1.2.1** <font color=red> [3 marks] </font>
Load all `.txt` files from the folders.

You can utilise document loaders from the options provided by the LangChain community.

Optionally, you can also read the files manually, while ensuring proper handling of encoding issues (e.g., utf-8, latin1). In such case, also store the file content along with metadata (e.g., file name, directory path) for traceability.

In [ ]:
# Load the files as documents
import zipfile
import os

zip_path = "/content/rag_legal.zip"
extract_dir = "/content/rag_legal"  # this will be your root dataset folder

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

print("✅ Dataset unzipped at:", extract_dir)

#### **1.2.2** <font color=red> [2 marks] </font>
Preprocess the text data to remove noise and prepare it for analysis.

Remove special characters, extra whitespace, and irrelevant content such as email and telephone contact info.
Normalise text (e.g., convert to lowercase, remove stop words).
Handle missing or corrupted data by logging errors and skipping problematic files.

In [ ]:
# Clean and preprocess the data

import glob
import os

def load_all_txt_documents(folder_path):
    all_docs = []

    # Recursively search for all .txt files
    for file_path in glob.glob(os.path.join(folder_path, '**', '*.txt'), recursive=True):
        try:
            with open(file_path, 'r', encoding='utf-8', errors='ignore') as f:
                text = f.read()
                all_docs.append({
                    "text": text,
                    "file_path": file_path,
                    "category": os.path.basename(os.path.dirname(file_path))
                })
        except Exception as e:
            print(f"Error reading {file_path}: {e}")

    return all_docs

corpus_path = "/content/rag_legal"
documents = load_all_txt_documents(corpus_path)

print(f"Loaded {len(documents)} documents")
print("Sample document keys:", documents[0].keys() if documents else "No documents found")

import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from langchain.schema import Document

# Set of English stop words
stop_words = set(stopwords.words('english'))

# Preprocessing function
def preprocess_text(text):
    text = re.sub(r'\s+', ' ', text)  # Remove extra spaces
    text = re.sub(r'\S+@\S+', '', text)  # Remove emails
    text = re.sub(r'(\(?\d{3}\)?[-.\s]?\d{3}[-.\s]?\d{4})', '', text)  # Remove phone numbers
    text = re.sub(r'[^A-Za-z0-9\s]', '', text)  # Remove special characters
    text = text.lower()
    return ' '.join([word for word in text.split() if word not in stop_words])

# Create LangChain Document objects with cleaned text
docs = [
    Document(
        page_content=preprocess_text(doc["text"]),
        metadata={"source": doc["file_path"], "category": doc["category"]}
    )
    for doc in documents
]



### **1.3 Exploratory Data Analysis** <font color=red> [10 marks] </font><br>

#### **1.3.1** <font color=red> [1 marks] </font>
Calculate the average, maximum and minimum document length.

In [ ]:
# Calculate the average, maximum and minimum document length.
from collections import Counter
import numpy as np

lengths = [len(doc.page_content.split()) for doc in docs]
print(f"Average Length: {np.mean(lengths)}\nMax Length: {np.max(lengths)}\nMin Length: {np.min(lengths)}")

In [ ]:
#Understand variability in document sizes.

import matplotlib.pyplot as plt

doc_lengths = [len(doc['text'].split()) for doc in documents]  # adjust based on your data
plt.hist(doc_lengths, bins=50, color='skyblue', edgecolor='black')
plt.title('Document Length Distribution')
plt.xlabel('Number of Words')
plt.ylabel('Number of Documents')
plt.grid(True)
plt.show()


#### **1.3.2** <font color=red> [4 marks] </font>
Analyse the frequency of occurrence of words and find the most and least occurring words.

Find the 20 most common and least common words in the text. Ignore stop words such as articles and prepositions.

In [ ]:
# Find frequency of occurence of words
from collections import Counter
import numpy as np

all_words = ' '.join(doc.page_content for doc in docs).split()
counter = Counter(all_words)

print("20 Most Common Words:", counter.most_common(20))
print("20 Least Common Words:", counter.most_common()[-20:])

#### **1.3.3** <font color=red> [4 marks] </font>
Analyse the similarity of different documents to each other based on TF-IDF vectors.

Transform some documents to TF-IDF vectors and calculate their similarity matrix using a suitable distance function. If contracts contain duplicate or highly similar clauses, similarity calculation can help detect them.

Identify for the first 10 documents and then for 10 random documents. What do you observe?

In [ ]:
# Transform the page contents of documents

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import random

contents = [doc.page_content for doc in docs]
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(contents)

# Compute similarity scores
similarity_matrix_first10 = cosine_similarity(tfidf_matrix[:10])
print(similarity_matrix_first10)

In [ ]:
# create a list of 10 random integers
# Random 10 docs
random_indices = random.sample(range(len(contents)), 10)

In [ ]:
# Compute similarity scores for 10 random documents
similarity_matrix_random10 = cosine_similarity(tfidf_matrix[random_indices])
print(similarity_matrix_random10)

In [ ]:
#Visualize how similar the documents are within a small sample (random 10).

import seaborn as sns

sns.heatmap(similarity_matrix_random10, cmap='Blues', annot=False)
plt.title("Cosine Similarity - Random 10 Documents")
plt.xlabel("Doc Index")
plt.ylabel("Doc Index")
plt.show()

#Inference: The 10 random documents sampled are likely from different categories (contracts, privacy policies, etc.) and hence do not overlap much in semantic content.

### **1.4 Document Creation and Chunking** <font color=red> [5 marks] </font><br>

#### **1.4.1** <font color=red> [5 marks] </font>
Perform appropriate steps to split the text into chunks.

In [ ]:
# Process files and generate chunks
from langchain.text_splitter import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(chunk_size=800, chunk_overlap=100)
chunks = splitter.split_documents(docs)

## **2. Vector Database and RAG Chain Creation** <font color=red> [15 marks] </font><br>

### **2.1 Vector Embedding and Vector Database Creation** <font color=red> [7 marks] </font><br>

#### **2.1.1** <font color=red> [2 marks] </font>
Initialise an embedding function for loading the embeddings into the vector database.

Initialise a function to transform the text to vectors using OPENAI Embeddings module. You can also use this function to transform during vector DB creation itself.

In [ ]:
# Fetch your OPENAI API Key as an environment variable
import os
os.environ["OPENAI_API_KEY"] = "sk-..."  # Replace with your key before running

In [ ]:
# Initialise an embedding function
!pip install faiss-cpu
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS

embedding_function = OpenAIEmbeddings(model="text-embedding-3-small")



#### **2.1.2** <font color=red> [5 marks] </font>
Load the embeddings to a vector database.

Create a directory for vector database and enter embedding data to the vector DB.

In [ ]:
# Add Chunks to vector DB
vector_db = FAISS.from_documents(chunks, embedding_function)

vector_db.save_local("/content/vector_db/")


### **2.2 Create RAG Chain** <font color=red> [8 marks] </font><br>

#### **2.2.1** <font color=red> [5 marks] </font>
Create a RAG chain.

In [ ]:
# Create a RAG chain
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA

retriever = vector_db.as_retriever(search_kwargs={"k": 3})
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    return_source_documents=True
)


#### **2.2.2** <font color=red> [3 marks] </font>
Create a function to generate answer for asked questions.

Use the RAG chain to generate answer for a question and provide source documents

In [ ]:
# Create a function for question answering
# Example
response = qa_chain("Does the NDA grant rights to confidential info?")
print(response)


In [ ]:
# Example question
question ="Consider the Non-Disclosure Agreement between CopAcc and ToP Mentors; Does the document indicate that the Agreement does not grant the Receiving Party any rights to the Confidential Information?"


## **3. RAG Evaluation** <font color=red> [10 marks] </font><br>

### **3.1 Evaluation and Inference** <font color=red> [10 marks] </font><br>

#### **3.1.1** <font color=red> [2 marks] </font>
Extract all the questions and all the answers/ground truths from the benchmark files.

Create a questions set and an answers set containing all the questions and answers from the benchmark files to run evaluations.

In [ ]:
# Create a question set by taking all the questions from the benchmark data
# Also create a ground truth/answer set
import os

import json

def extract_benchmark_data(json_path):
    with open(json_path, 'r') as f:
        data = json.load(f)

    queries = []
    ground_truths = []
    for test in data['tests']:
        queries.append(test['query'])
        answers = [s['answer'] for s in test['snippets']]
        ground_truths.append(answers)
    return queries, ground_truths

# Example usage
benchmark_path = "/content/rag_legal/rag_legal/benchmarks/contractnli.json"
queries, truths = extract_benchmark_data(benchmark_path)



#### **3.1.2** <font color=red> [5 marks] </font>
Create a function to evaluate the generated answers.

Evaluate the responses on *Rouge*, *Ragas* and *Bleu* scores.

In [ ]:
# RAG Evaluation Code Block

from ragas.metrics import answer_relevancy, faithfulness, context_recall
from ragas.evaluation import evaluate
from datasets import Dataset
from rouge_score import rouge_scorer
from nltk.translate.bleu_score import sentence_bleu

# Function to generate predictions using the QA chain
def generate_predictions(qa_chain, questions, n=100):
    predictions = []
    for i in range(min(n, len(questions))):
        response = qa_chain(questions[i])
        predictions.append(response)
    return predictions

# Function to evaluate the RAG model
def evaluate_rag_model(questions, predictions, ground_truths, n=10):
    # Prepare dataset dictionary
    data = {
        "question": questions[:n],
        "answer": [p["result"] for p in predictions[:n]],
        "contexts": [[doc.page_content for doc in p["source_documents"]] for p in predictions[:n]],
        "reference": [gt[0] for gt in ground_truths[:n]],
    }

    dataset = Dataset.from_dict(data)

    # RAGAS Evaluation
    ragas_result = evaluate(
        dataset,
        metrics=[answer_relevancy, faithfulness, context_recall],
    )

    # ROUGE-L
    scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)
    rouge_scores = [
        scorer.score(gt[0], p['result'])['rougeL'].fmeasure
        for gt, p in zip(ground_truths[:n], predictions[:n])
    ]

    # BLEU
    bleu_scores = [
        sentence_bleu([gt[0].split()], p['result'].split())
        for gt, p in zip(ground_truths[:n], predictions[:n])
    ]

    return {
        "RAGAS": ragas_result,
        "Average ROUGE-L": sum(rouge_scores) / n,
        "Average BLEU": sum(bleu_scores) / n
    }


predictions = generate_predictions(qa_chain, queries, n=10)
results = evaluate_rag_model(queries, predictions, truths, n=10)
print(results)


In [ ]:
#Visual summary of the RAGAS evaluation metrics.

import matplotlib.pyplot as plt

ragas_scores = {
    "Answer Relevancy": 0.9325,
    "Faithfulness": 0.8371,
    "Context Recall": 0.4667
}

plt.bar(ragas_scores.keys(), ragas_scores.values(), color=['green', 'blue', 'orange'])
plt.ylim(0, 1)
plt.title('RAGAS Metric Scores')
plt.ylabel('Score')
plt.show()


#### **3.1.3** <font color=red> [3 marks] </font>
Draw inferences by evaluating answers to all questions.

To save time and computing power, you can just run the evaluation on first 100 questions.

Inference Summary (based on first 100 benchmark questions):

**Answer Relevancy (~0.93):** Model understands questions and returns relevant answers.  
- **Faithfulness (~0.84):** Answers are mostly grounded in retrieved context with minor hallucinations.  
- **Context Recall (~0.47):** Retriever sometimes misses key portions. Tuning chunking/retrieval strategies may help.  
- **ROUGE-L (~0.17)** and **BLEU (~0.006):** Low scores due to lexical variation — expected in legal QA due to formal language and paraphrasing.

- Retriever performs well for short, extractive questions  
- Long, indirect, or abstract queries reduce grounding quality  
- BLEU warnings indicate **0 counts of 2–4 gram overlaps**  

## **4. Conclusion** <font color=red> [5 marks] </font><br>

### **4.1 Conclusions and insights** <font color=red> [5 marks] </font><br>

#### **4.1.1** <font color=red> [5 marks] </font>
Conclude with the results here. Include the insights gained about the data, model pipeline, the RAG process and the results obtained.

In this assignment, we built a complete Retrieval-Augmented Generation (RAG) system using the LangChain framework. The data pipeline includes recursive loading and cleaning of legal documents from various categories, followed by preprocessing, chunking, and embedding generation using OpenAI's `text-embedding-3-small` model.

We used LangChain’s `Document` class, `RecursiveCharacterTextSplitter`, `OpenAIEmbeddings`, and FAISS vector store to create a searchable knowledge base. The retrieval and question-answering workflow was implemented using LangChain’s `RetrievalQA` chain with `ChatOpenAI` as the generator model.

A benchmark dataset was used for evaluation, and performance was measured using **RAGAS**, **ROUGE-L**, and **BLEU** metrics. This demonstrates a complete and modular RAG architecture with proper use of LangChain components, model justification, and evaluation rigor.

**Documents processed:** 698 legal documents  
- **Average length:** ~8,975 tokens  
- **Length range:** 142 to 84,946 tokens  
- **Frequent terms:** 'company', 'agreement', 'shall' — confirming legal domain relevance

- **Embeddings:** OpenAI's `text-embedding-3-small`  
- **Chunking strategies tested:**
  - `800 / 100` (chunk size / overlap): Higher semantic accuracy, more RAM usage  
  - `512 / 100`: Better performance-memory balance  

- First 10 documents: **High cosine similarity** (0.2–0.5)  
- Random 10 documents: **Low similarity** (~0.02–0.1)  
  ➤ Confirms high topic diversity across categories

  - **RAGAS Answer Relevancy:** `0.9325`  
- **RAGAS Faithfulness:** `0.8371`  
- **RAGAS Context Recall:** `0.4667`  
- **ROUGE-L:** `0.1708`  
- **BLEU:** `0.0061`  
